### Experiment 4 - **NOT STARTED**
- Test the ability of a model trained on Dataset 1 to generalize to Dataset 3
- Negative Class (0) Non-Cyberbullying
- Positive Class (1) Cyberbullying

In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from pathlib import Path
import json
import random
import sys
sys.path.append('C:\\Users\\rooty\\UWEC\\Research\\CyberBullyingML\\venv\\cyberbullying-ml\\src')
from utils.results import create_results_file, append_results_to_json
from utils.data import balance_train_and_test
from catboost import CatBoostClassifier


In [3]:
RANDOM_SEED = 115
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

In [6]:
CLEAN_DATA_PATH = Path("../data/en_only")

EXPERIMENTS_PATH = Path('C:\\Users\\rooty\\UWEC\\Research\\CyberBullyingML\\venv\\cyberbullying-ml\\experiments')
RESULT_PATH = Path('C:\\Users\\rooty\\UWEC\\Research\\CyberBullyingML\\venv\\cyberbullying-ml\\experiments\\results\\exp4')

In [24]:
TRAIN_DATA_NAME = '48000_cyberbullying_tweets_basic_clean.csv'
TEST_DATA_NAME = 'onlineHarassmentDataset_basic_clean.csv'

train_data = CLEAN_DATA_PATH / TRAIN_DATA_NAME
test_data = CLEAN_DATA_PATH / TEST_DATA_NAME

In [27]:
train_df = pd.read_csv(train_data)
test_df = pd.read_csv(test_data)

test_df.rename(columns={'Code': 'label', 'Tweet': "tweet"}, inplace=True)

train_df['label'] = train_df['label'].apply(lambda x : 0 if x == 'notcb' else 1)
test_df['label'] = test_df['label'].apply(lambda x : 0 if x == 'N' else 1)

In [28]:
train_df.dropna(axis=0, inplace=True)
train_df.drop_duplicates(inplace=True)
train_df.reset_index(drop=True, inplace=True)
train_df['label'].value_counts()

label
1    37170
0     6377
Name: count, dtype: int64

In [29]:
test_df.dropna(axis=0, inplace=True)
test_df.drop_duplicates(inplace=True)
test_df.reset_index(drop=True, inplace=True)
test_df['label'].value_counts()

label
0    14202
1     4855
Name: count, dtype: int64

In [31]:
# balance the datasets
notcb_train, cb_train = train_df[train_df['label'] == 0], train_df[train_df['label'] == 1]
notcb_test, cb_test = test_df[test_df['label'] == 0], test_df[test_df['label'] == 1]
tr_df, te_df, ratio = balance_train_and_test(notcb_train, cb_train, notcb_test, cb_test, random_state=RANDOM_SEED)

In [35]:
print(notcb_train.shape[0] / cb_train.shape[0])
print(notcb_test.shape[0] / cb_test.shape[0])
ratio

0.17156308851224106
2.925231719876416


0.48968471148126125